In [ ]:
# On Colab, you can uncomment the line below to install the required packages.
# !pip install networkx igraph pandas numpy matplotlib seaborn scipy

# Clone the repo (to get access to the module)
#!git clone https://github.com/jgarciab/NetworkScience.git
#import sys
#sys.path.append('/content/NetworkScience/Practicals/day1/')


In [ ]:
# on your computer
path_data = "../../Data/"

# on colab uncomment the line below
# path_data = "/content/NetworkScience/Data/"

# Introduction to Network Science Practical

Welcome to the first practical session on network science! In this tutorial, you will learn how to read, visualize, and analyze real-world networks using Python libraries such as `networkx`, `igraph`, and `pandas`.

**Learning objectives:**
- Get comfortable with basic network data structures and visualization.
- Explore key network properties (connectedness, diameter, density, clustering, assortativity, degree distribution, robustness).
- Compare real-world networks to reference models.

Each section contains explanations, code, and exercises. Read the explanations carefully and try to understand the code before running it. Comments in the code will help guide you.

In [ ]:
# Import network analysis libraries
import networkx as nx  # For network analysis
import igraph as ig    # For fast network analysis and layout

# Import data processing libraries
import pandas as pd    # For data manipulation
import numpy as np     # For numerical operations

# Import utility functions from local file
import sys
sys.path.append("./")
from common_functions import *  # Custom helper functions

# Import visualization libraries
import pylab as plt
import seaborn as sns

# Set custom visualization parameters for better aesthetics
custom_params = {
    "axes.spines.right": False, "axes.spines.top": False, "axes.spines.left": False, "axes.spines.bottom": False,
    "lines.linewidth": 2, "grid.color": "lightgray", "legend.frameon": False,
    "xtick.labelcolor": "#484848", "ytick.labelcolor": "#484848",
    "xtick.color": "#484848", "ytick.color": "#484848",
    "text.color": "#484848", "axes.labelcolor": "#484848",
    "axes.titlecolor":"#484848", "figure.figsize": [5,3],
    "axes.titlelocation":"left",
    "xaxis.labellocation":"left",
    "yaxis.labellocation":"bottom"
}
# Define a color palette for plots
palette = ["#3d348b","#e6af2e","#191716","#e0e2db"]  # Custom color palette
sns.set_theme(context='paper', style='white', palette=palette, font_scale=1.3, color_codes=True, rc=custom_params)

# Widen the notebook display for better plot visibility
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Exercise 1: Reading and visualizing graphs

**Goal:** Get used to the `networkx` library for network analysis and visualization.

In this section, you'll learn how to load example networks, inspect their structure, and visualize them.

## 1.1. Basic plot
- Read and understand the following code

In [ ]:
# Read an example data on florentine marriage families in the XV century
G = nx.florentine_families_graph()

# Show the nodes and edges
print("Nodes: ", G.nodes())
print("Edges: ", G.edges())


In [ ]:
# Visualize it
plt.figure(figsize=(5,4)) #set up dimensions

# create layout (once so we can reuse it if needed)
# the layout keeps the coordinates of all labels
pos = nx.spring_layout(G, seed = 1)
nx.draw(G, pos = pos, with_labels = True, 
        edge_color = "gray", node_color = "lightgray")

## 1.2. Read and visualize your network
Network here: https://tinyurl.com/network-game
(I added the direct CSV link. But you could also download it as a CSV file and move it to the folder of this notebook) 

Use the `nx.from_pandas_edgelist` function (you need to set up the `source` and `target` parameters)
Print number of nodes and edges

In [ ]:
# Read edgelist (link)
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSuZC86KjXYKSPr0Nw7mfRha4zwea3aMw-gTKliVcbRt_m3NRiCEyxbH_d5M8MBL0LWayg1WDmnqBET/pub?gid=0&single=true&output=csv")
df.head()

In [ ]:
# Get the documentation of the function to convert a pandas DataFrame to a NetworkX graph
nx.from_pandas_edgelist?

In [ ]:
# Convert to networkx (replace "..." with the actual DataFrame variable name)
G = nx.from_pandas_edgelist(..., source="P1", target="P2")
print(len(G.nodes()))
print(len(G.edges()))

In [ ]:
# create figure and plot
plt.figure(figsize=(10,8))

# create layout (once so we can reuse it if needed)
pos = nx.spring_layout(G, seed = 1)
nx.draw(G, pos = pos, with_labels = True, node_size=10,
        edge_color = "lightgray", node_color = "gray")

## 1.3a. Read and visualize the Twitter network

We will be using (parts of) this network in the following days.

It contains mentions, replies and quotes of user A to user B (but let's treat is as undirected)
```
replied_to    40255
mentioned     10407
quoted         3448
```

Use the `nx.from_pandas_edgelist` function
Print number of nodes and edges

In [ ]:
# Read edgelist
df = pd.read_csv(f"{path_data}/ic2s2_netsci_3.tsv", sep="\t")
display(df.head())

# Convert to networkx and remove self-edges
G_netsci = nx.from_pandas_edgelist(df)
G_netsci.remove_edges_from(nx.selfloop_edges(G_netsci)) #remove self-edges
print(len(G_netsci.nodes()))
print(len(G_netsci.edges()))

In [ ]:
# create figure and plot
plt.figure(figsize=(20,16))

# create layout (once so we can reuse it if needed)
pos = nx.spring_layout(G_netsci, seed = 1)
nx.draw(G_netsci, pos = pos, with_labels = False, node_size=10,
        edge_color = "lightgray", node_color = "gray")

## 1.3b. Read and visualize the Protein-Protein interaction network

We will be using (parts of) this network in the following days.

It contains recorded interactions between pair of proteins in yeas (_S. cerevisiae_).

The network data is in a `.graphml` file, a very commonly used network format.

Use the `nx.read_graphml` function
Print number of nodes and edges

In [ ]:
# Read PPI network
path_network = f"{path_data}/ppi_network_prediction.graphml"
G_ppi = nx.read_graphml(path_network)
len(G_ppi.nodes()), len(G_ppi.edges())


In [ ]:
# igraph is faster. Here is how to convert to igraph
h = ig.Graph().from_networkx(G_ppi)
# create layout
layout = h.layout_fruchterman_reingold()
# create positions in the networx format (as a dictionary)
pos = dict(zip(G_ppi.nodes(), layout.coords))

# Plot as before
plt.figure(figsize=(12, 10))
nx.draw(G_ppi, pos = pos, with_labels = False, node_size=20,
        edge_color = "lightgray", node_color = "gray")

## 1.4. Read and visualize a large(r) network 
- Read network in f"{path_data}/wiki-Vote.txt". Careful, it is a directed network, you need to use the create_using parameter.

(use iterations = 30 in the spring_layout)

Format of the file:
```
# Directed graph (each unordered pair of nodes is saved once): Wiki-Vote.txt 
# Wikipedia voting on promotion to administratorship (till January 2008). Directed edge A->B means user A voted on B becoming Wikipedia administrator.
# Nodes: 7115 Edges: 103689
# FromNodeId	ToNodeId
30	1412
30	3352
```

In [ ]:
# Read directed graph
G_wiki = nx.read_edgelist(f"{path_data}/wiki-Vote.txt", create_using=nx.DiGraph())
print(len(G_wiki.nodes()))
print(len(G_wiki.edges()))

In [ ]:
# Create layout (this will take a couple minutes). Networkx is a particularly slow library
pos = nx.spring_layout(G_wiki, seed = 1, iterations=30)

In [ ]:
# Nobody wants to see your hairball, but let's plot it anyway
plt.figure(figsize=(20, 20))

# Plot only nodes (too many lines)
nx.draw_networkx_nodes(G_wiki, pos = pos, node_size = 1, node_color = "k")


# Exercise 2: Network characteristics

Network example: Protein interaction (PPI) in yeast. Nodes = Proteins, Edges = Recorded interactions in experiments.

**Exercise:** Compare the PPI network and the Twitter network (ic2s2_netsci_3). What characteristics apply to both?

In this section, you'll learn how to compute and interpret key network properties.

In [ ]:
# Create a copy of the PPI graph for further analysis (change this to G_netsci to study the Twitter network or G_wiki for the Wikipedia network)
G = G_ppi.copy()  

In [ ]:
# igraph is faster. Here is how to convert to igraph
h = ig.Graph().from_networkx(G)
# create layout
layout = h.layout_fruchterman_reingold()
# create positions in the networx format (as a dictionary)
pos = dict(zip(G.nodes(), layout.coords))

# Plot as before
fig, ax = plt.subplots(figsize=(10, 8))
#This function is defined in common_functions.py file. It adds automatic coloring and nicer defaults
plot_network(G, a0 = ax, pos = pos, with_labels=False) 

## 2.1 Connectedness

- A connected component is a set of nodes where each node is reachable from any other.
- Many real-world networks are not fully connected.
- The largest connected component is often called the 'giant component'.

In [ ]:
# Number of connected components
nx.components.number_connected_components(G)

In [ ]:
# Size of the connected components (first 20)
sorted([len(_) for _ in nx.connected_components(G)])[:-20:-1]

In [ ]:
# Let's keep the largest component (be careful, don't do this in your data withuot a very good reason)
largest_cc = max(nx.connected_components(G), key=len)

# Create a subgraph of G consisting only of nodes in largest_cc
G = G.subgraph(largest_cc).copy()
len(G)

## 2.2 Small diameters

- The diameter is the longest shortest path between any two nodes.
- The average shortest path length is the mean of all shortest paths.
- Small diameters are typical in social and biological networks.

In [ ]:
# Diameter
nx.diameter(G)

In [ ]:
# Average shortest path
nx.average_shortest_path_length(G)

## 2.3 Density

- Density is the ratio of actual edges to possible edges.
- Sparse networks have low density.

In [ ]:
nx.density(G)

## 2.4 Transitivity and average clustering 

- Clustering measures the tendency of nodes to form triangles (closed triplets).
- Transitivity is the global clustering coefficient.
- Average clustering is the mean of local clustering coefficients.

In [ ]:
# Average at the network level (# triangles / possible number of triangles)
print(nx.transitivity(G))

# Average at the node level = Average(# triangles_i / possible triangles_i )
print(nx.average_clustering(G))

## 2.5 Assortativity and homophily

- Assortativity measures the tendency of nodes to connect to similar nodes (e.g., by degree or attribute).
- Positive assortativity: high-degree nodes connect to other high-degree nodes.
- Negative assortativity: high-degree nodes connect to low-degree nodes.

In [ ]:
nx.assortativity.degree_assortativity_coefficient(G)

## 2.6 Heavy tails

- Many real-world networks have a few nodes with very high degree (hubs) and many with low degree.
- This is called a heavy-tailed or scale-free distribution.
- Plotting the degree distribution helps reveal this property.

In [ ]:
degree_list = dict(G.degree()).values()
from collections import Counter

# count the number of nodes with a specific degree, sort it
C = Counter(degree_list)
deg, cnt = zip(*sorted(C.items()))

# Plot
plt.plot(deg, cnt, "o-")
plt.xlabel("Degree")
plt.ylabel("Count (degree)")


In [ ]:
# Defined in common_functions.py
plot_cdf(degree_list, compl = True, xlabel = "Degree (d)", ylabel = "p(Degree > d)")


## 2.7 Robustness to failures / Fragility to targeted attacks

- Networks can be robust to random failures but fragile to targeted attacks (removing hubs).
- The code below simulates both scenarios and plots the effect on the largest component.
- Compare the behavior of real and random networks.

In [ ]:
def attack_network(G, how="targeted", n_remove=200):
    """
    Removes nodes iteratively and keeps the (harmonic average of distance between all pairs of nodes)
    
    how = "targeted" targets the node with the highest degree
    """
    n_components = []

    G2 = G.copy()
    for i in range(n_remove):
        if how == "targeted":
            # Find the node with the highest degree
            node_to_remove = max(G2.degree(), key=lambda x: x[1])[0]

        else:
            # Random node
            node_to_remove = np.random.choice(G2.nodes(), 1)[0]

        # Remove node
        G2.remove_node(node_to_remove)
        # Number of nodes in giant component
        if isinstance(G2, nx.DiGraph):
            n_comp = max([len(_) for _ in nx.weakly_connected_components(G2)])
        else:
            n_comp = max([len(_) for _ in nx.connected_components(G2)])
        n_components.append(100*n_comp/len(G2))
        
        
    return n_components

In [ ]:
# Create an ER graph (random graph)
G_r = nx.random_graphs.gnm_random_graph(1000, len(G.edges()))
# Let's keep the largest component (be careful, don't do this in your data withuot a very good reason)
largest_cc = max(nx.connected_components(G_r), key=len)

# Create a subgraph of G consisting only of nodes in largest_cc
G_r = G_r.subgraph(largest_cc).copy()

# Plot network random
plot_network(G_r, with_labels=False)
plt.show()

# Plot network PPI
plot_network(G, with_labels=False)

In [ ]:
n_remove = 200
n_comp_attack_t = attack_network(G, how="targeted", n_remove = n_remove)
n_comp_attack_r = attack_network(G, how="random", n_remove = n_remove)

ref_n_comp_attack_t = attack_network(G_r, how="targeted", n_remove = n_remove)
ref_n_comp_attack_r = attack_network(G_r, how="random", n_remove = n_remove)

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(100*np.arange(n_remove)/len(G), n_comp_attack_t, label="Protein Interaction")
plt.plot(100*np.arange(n_remove)/len(G), ref_n_comp_attack_t, label="Random network")
plt.xlabel("Share of nodes removed")
plt.ylabel("Share of nodes in giant component")
plt.legend()
plt.title("Targeted attack")
plt.ylim(0,100)

plt.subplot(122)
plt.plot(100*np.arange(n_remove)/len(G), n_comp_attack_r, label="Protein Interaction")
plt.plot(100*np.arange(n_remove)/len(G), ref_n_comp_attack_r, label="Random network")
plt.xlabel("Share of nodes removed")
plt.ylabel("Share of nodes in giant component")
plt.legend()
plt.title("Random attack")
plt.ylim(0,100)

### Comparison PPI and Twitter
- Connectedness:
    - Twitter: 
    - PPI: 
- Diameter:
    - Twitter: 
    - PPI: 
- Density,
    - Twitter: 
    - PPI: 
- Transitivity:
    - Twitter: 
    - PPI: 
- Average clustering:
    - Twitter: 
    - PPI: 
- Degree assortativity:
    - Twitter: 
    - PPI: 
- Robustness:
    - Twitter: 
    - PPI: 
    

# Exercise 3: Distributions (i.e. looking at characteristics of the nodes)
- Degree
- Number of triangles
- Clustering (transitivity)
- Local assortativity (homophily)
- Path length

__Goal__: Understand how to calculate distributions in `networkx`, interpret differences in the degree distribution 

__Exercise__: In 3.7 compare the Wikipedia network (default example), the PPI network and the Twitter network (IC2S2)

In [ ]:
# Read data on florentine marriage families in the XV century
G = nx.florentine_families_graph()
len(G)

In [ ]:
# Use the following function to plot the CDF of the degree distributions
def plot_cdf(values, scale = "log", ax = None, cum = True, compl = False, marker = 'o-', xlabel = "Degree (d)", ylabel = "p(Degree < d)"):
    """
    Calculates and plot CDF
    """
    
    from collections import Counter

    # count the number of instance per each degree, sort it
    C = Counter(values)
    deg, cnt = zip(*sorted(C.items()))
    
    # calcualte the cumulative distribution, normalize to be a probability instead of a count
    if cum:
        cs = np.cumsum(cnt)/np.sum(cnt)
    else:
        cs = cnt/np.sum(cnt)
    
    if compl:
        cs = 1 - cs
        
    if ax is None:
        ax = plt.subplot()
    # plot
    ax.plot(deg, cs, marker)
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    plt.tight_layout()
    sns.despine(left=True, bottom=True)
    plt.xscale(scale)
    plt.yscale(scale)
    

## 3.1 Degree distribution

- The degree of a node is the number of connections it has.
- Plotting the degree distribution helps identify hubs and the overall structure.
- Use both histograms and cumulative distribution functions (CDF/CCDF) for insight.

In [ ]:
def plot_network_distribution(G, values, mult = 1000, with_labels = True):
    """
    Plots network (color and node size depends on values) and distributions
    """
    import matplotlib as mpl
    norm = mpl.colors.Normalize(vmin=min(values), vmax=max(values), clip=True)
    mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)


    f, (a0, a1, a2) = plt.subplots(1, 3, gridspec_kw={'width_ratios': [2, 1, 1]}, figsize=(12,4))
    
    node_size = mult*np.array(list(values))
    if min(node_size) < 0:
        node_size -= min(node_size)
        node_size += 1
        
    nx.draw(G, pos = nx.spring_layout(G, seed = 1), with_labels = with_labels, node_size = node_size, edge_color = "gray", 
           node_color = [mapper.to_rgba(i) for i in values], ax = a0,)


    sns.histplot(values, ax = a1)
    
    
    plot_cdf(values, ax = a2, compl = True, xlabel = "Cent c", ylabel = "p(Cent > c)")

In [ ]:
# Degree distribution
degree = G.degree() #also nx.degree(G)
degree_values = dict(degree).values()

# Plot using sns.histplot
sns.histplot(degree_values)
print(sorted(degree_values))

In [ ]:
# Plot CDF in log-log scale
plot_cdf(degree_values, scale = "log", ax = None, cum = True, compl = True)

In [ ]:
# change "mult" to change the size of the nodes 
plot_network_distribution(G, degree_values, mult = 100)

## 3.2 Distribution of number of triangles

- The number of triangles a node participates in is a measure of local clustering.
- Plotting this distribution shows which nodes are in tightly-knit groups.

In [ ]:
# Distribution of the number of triangles per node
n_triangs = nx.triangles(G).values() 
display(list(zip(G.nodes(), n_triangs))[:20])

plot_network_distribution(G, n_triangs, mult = 100)

## 3.3 Distribution of clustering

- Clustering measures the tendency of nodes to form tightly-knit groups.
- This distribution shows how many clusters (triangles) each node is part of.

In [ ]:
# Replace "..." with the actual calculation of clustering coefficients (nx....)
n_clus = ...
display(list(zip(G.nodes(), n_clus.values()))[:20])

plot_network_distribution(G, n_clus, mult = 1000)

### 3.4 Distribution of shortest paths

- The distribution of shortest path lengths gives insight into how 'small' the network is.
- For large networks, calculating all shortest paths can be slow.
- Consider sampling or using approximate methods for very large graphs.

In [ ]:
# Calculate all shortest paths (careful, this quickly becomes unfeasible)
path_lenghts = ...

# Get results from a nested dictionary
path_lenghts = [list(_[1].values()) for _ in path_lenghts]
path_lenghts = [subitem for item in path_lenghts for subitem in item ]

# Plot using sns.histplot
sns.histplot(path_lenghts)
plt.show()

## 3.5 Distribution of local assortativity

- Local assortativity measures how similar a node's neighbors are to itself.
- This can reveal patterns of homophily (like-with-like) at the node level.

In [ ]:
attribute = [k for v,k in G.degree()]
# Defined in common_functions (based on Peel et al 2018)
local_assort = calculate_local_assort(G, attribute)
display(list(zip(G.nodes(), local_assort))[:20])

plot_network_distribution(G, local_assort, mult = 1000)

## 3.6 Degree distribution if ~ normal distribution (used in the lectures)


In [ ]:
# Don't change this
def plot_distributions(d, scale="log"):
    """
    Plot the PDF, CDF and CCDF.
    """
    plt.figure(figsize=(12,4))
    ax = plt.subplot(131)
    plot_cdf(d, cum = False, ax = ax,  xlabel = "Degree (d)", ylabel = "p(Degree)", marker = ".", scale=scale)

        
    plt.title("PDF")
    ax = plt.subplot(132)
    plot_cdf(d, cum = True, ax = ax,  xlabel = "Degree (d)", ylabel = "p(Degree < d)", marker="-", scale=scale)
    plt.title("CDF")
    ax = plt.subplot(133)
    plot_cdf(d, compl = True, ax = ax,  xlabel = "Degree (d)", ylabel = "p(Degree > d)", marker=".-", scale=scale)
    plt.title("CCDF")
    

In [ ]:
## Degree distribution (random normally distributed data)
## Try changing the scale to "log"
d = np.random.binomial(500, p = 30/500, size = 10000)
plot_distributions(d, scale="linear")


## 3.7 Degree distribution in the Wiki/Twitter/PPI networks

- Compare the degree distributions of the Wikipedia, Twitter, and PPI networks.
- Use log-log plots to highlight heavy tails.
- What differences do you observe?

In [ ]:
## Degree distribution (wiki network)
# Try the code using scale = "linear"
G_wiki = nx.read_edgelist(f"{path_data}/wiki-Vote.txt", create_using=nx.DiGraph())
d = [v for k,v in G_wiki.degree()]
plot_distributions(d, "log")
plt.xlim([100,1000])
plt.plot([2E2, 1E3], [2.5E-2,2.5E-4],"--")



In [ ]:
# Twitter data
df = pd.read_csv(f"{path_data}/ic2s2_netsci_3.tsv", sep="\t")
G_twitter = nx.from_pandas_edgelist(df)
G_twitter.remove_edges_from(nx.selfloop_edges(G_twitter)) #remove self-edges

d = [v for k,v in G_twitter.degree()]
plot_distributions(d, "log")
plt.xlim([100,1000])
plt.plot([2E2, 1E3], [2.5E-2,2.5E-4],"--")

In [ ]:
# PPI data
path_network = f"{path_data}/ppi_network_prediction.graphml"
G_ppi = nx.read_graphml(path_network, node_type=int)

d = [v for k,v in G_ppi.degree()]
plot_distributions(d, "log")

plt.plot([2E1, 1E2], [2.5E-2,2.5E-4],"--")

# Exercise 4
Is the network homophilic? (Do nodes connect to similar nodes?)

- Calculate the assortativity coefficient.
- Use permutation tests and random models to assess significance.

**Goal:** Understand permutation tests (4A) and reference models (4B).

## Exercise 4A: Permutation test

- Shuffle node labels and recalculate assortativity to create a null distribution.
- Compare the observed value to the null to assess significance.
- The p-value is the probability of observing a value as extreme as the real one under the null.

In [ ]:
# Create labels (I made them up)
ns = ['Acciaiuoli', 'Medici', 'Castellani', 'Peruzzi', 'Strozzi', 'Barbadori', 'Ridolfi', 'Tornabuoni', 'Albizzi', 'Salviati', 'Pazzi', 'Bischeri', 'Guadagni', 'Ginori', 'Lamberteschi']
classes = ["m", "m", "o", "o", "o", "o", "o", "m", "m", "m", "o", "o", "o", "o", "o"]
nx.set_node_attributes(G, dict(zip(ns,classes)), "classes")

# Plot
nx.draw(G, pos = nx.spring_layout(G, seed = 1), with_labels = True, edge_color = "gray", 
       node_color = [palette[0] if c == "m" else palette[1] for c in classes])
    

In [ ]:
# calculate assortativity coefficient
assort = ...
assort

In [ ]:
# Permute node labels
G2 = G.copy()

# Randomize classes and calculate assortativity
iters = 10000
values = []
for i in range(iters):
    # shuffle the classes
    permuted_classes = np.random.permutation(classes)
    # map classes to nodes 
    map_classes = dict(zip(ns, permuted_classes))
    nx.set_node_attributes(G2, map_classes, "classes")
    # calculate assortativity and keep in a list
    values.append(nx.assortativity.attribute_assortativity_coefficient(G2, "classes"))
values = np.array(values)

# Plot results
sns.histplot(values, binwidth=0.05)
plt.plot([assort, assort],[0, iters/10], "--", color="gray")

# p-value (probability that we would observe a value equal or more extreme to the one observed given 
# that the null hyphotesis is true---i.e. the graph is the real graph and the links 
# are not correlated with the classes
1-len(values[values<assort])/len(values)

# Exercise 4B: Reference model
Is the network transitive? (are your friends also friends themselves?)

We can calculate the average clustering

- Problem:  We have nothing to compare against.
- Solution: Shuffle the edges (permutation model) or Grow the network (generative model)


In [ ]:
# Read data on florentine marriage families in the XV century
G = nx.florentine_families_graph()

In [ ]:
# Calculate average clustering
print(...)
# Calculate global clustering (transitivity)
print(...)
# Calculate degree assortativity
print(...)

#### We can compare our network to randomly generated data (this is useful to test hypothesis/get a baseline)

In [ ]:
# Visualize random graphs (run several times)
degree_seq = [v for k,v in G.degree()]
G_r = nx.configuration_model(degree_seq)
G_r = nx.Graph(G_r)
nx.draw(G_r)

In [ ]:
# Visualize random graphs (run several times)
n = len(G)
m = len(G.edges())
G_r = nx.random_graphs.barabasi_albert_graph(n,int(m/n)+1)
G_r = nx.Graph(G_r)
nx.draw(G_r)

In [ ]:
# Visualize random graphs (run several times)
n = len(G)
m = len(G.edges())
G_r = nx.random_graphs.gnm_random_graph(n,m)
G_r = nx.Graph(G_r)
nx.draw(G_r)

#### The function conf_int (defined in common_functions.py) creates 100 random graphs and calculates the desired metric

In [ ]:
# Average local clustering coefficient
conf_int(G, nx.average_clustering, 100)
nx.average_clustering(G)
#same as np.mean(list(nx.clustering(G).values()))

In [ ]:
# Transitivity (global clustering)
conf_int(G, nx.transitivity, 100)
nx.transitivity(G)

In [ ]:
# Assortativity
conf_int(G, nx.assortativity.degree_assortativity_coefficient, 100)
nx.assortativity.degree_assortativity_coefficient(G)


# Exercise 4B.2

Use igraph to read the network and calculate assortativity, and compare with random model (ER)

# In this section, you'll see how to use igraph for fast network analysis and compare real network properties to those of random graphs.
#
# This helps you understand if observed properties are due to network structure or just random chance.

In [ ]:
G_wiki = nx.read_edgelist(f"{path_data}/wiki-Vote.txt", create_using=nx.DiGraph())
# Read directed graph and print basic info
print(len(G_wiki.nodes()))  # Number of nodes
print(len(G_wiki.edges()))  # Number of edges

# Convert to igraph for faster computation
h = ig.Graph.from_networkx(G_wiki)

# Compare speed of assortativity calculation in igraph vs networkx
%timeit h.assortativity_degree()  # igraph: fast
%timeit nx.assortativity.degree_assortativity_coefficient(G_wiki)  # networkx: slower

### Compare with random model (ER)

- Compare your network's clustering and assortativity to random graphs with the same degree sequence.
- Use the configuration model or other random graph models.
- This helps determine if observed properties are due to chance or network structure.

In [ ]:
# networkx
def conf_dens_nx(n, m):
    """Create random graph and calculate assortativity in networkx"""
    G_r = nx.random_graphs.gnm_random_graph(n,m)
    return nx.assortativity.degree_assortativity_coefficient(G_r)
def conf_dens_ig(n,m):
    """Create random graph and calculate assortativity in igraph"""
    h_r = ig.Graph.Erdos_Renyi(n=n,m=m)
    return h_r.assortativity_degree()

In [ ]:
%%time
# Doing it in networkx 10 times
n, m = len(G_wiki), len(G_wiki.edges())
print(np.percentile([conf_dens_nx(n, m) for i in range(10)], [5,95]))

nx.assortativity.degree_assortativity_coefficient(G_wiki)

In [ ]:
%%time
# Doing it in igraph 100 times
n, m = h.vcount(), h.ecount()
print(np.percentile([conf_dens_ig(n,m) for i in range(100)], [5,95]))

h.assortativity_degree()